In [1]:
import os
import pandas as pd
import math
import random

In [2]:
from termcolor import colored

SPADES = f'\u2660'  
CLUBS = '\u2663'
HEARTS = colored('\u2665', 'red')
DIAMONDS = colored('\u2666', 'red')

In [ ]:
def _split(total):
    '''return every possible 2 number tuple that sums to total'''
    combos = []
    for i in range(total):
        rem = total - i
        combos.append((i, rem))
    return combos

def every_split_in_3(total):
    ''' Returns Every possible set of 3 numbers that add up to total'''
    all_distribs = []
    two = _split(total)
    for s in two:
        three = _split(s[1])
        for s3 in three:
            all_distribs.append((s[0], s3[0], s3[1] ))
    return(all_distribs)

def every_split_in_4(total):
    ''' Returns Every possible set of 4 SUIT distribution that add up to total'''
    all_distribs = []
    two = _split(total)
    for s in two:
        three = _split(s[1])
        for s3 in three:
            four = _split(s3[1])
            for s4 in four:
                all_distribs.append((s[0], s3[0], s4[0],  s4[1] ))
    return(all_distribs)

In [4]:
def generate_points_single_hand(score):
    '''Returns Four numbers (HCPs) that add up to a given total score'''

    if score > 40:
        raise ValueError(f"Points {score} should be less than 40 per hand")
    

    pts = [0 , 0 , 0 , 0]
    remainder = score
    for s in range(3):        
        if remainder > 0:
            pts[s] = random.randint(0, min(remainder, 10))
            remainder = remainder - pts[s]
    pts[3] = remainder
    
    return(pts)

In [5]:
generate_points_single_hand(18)

[0, 6, 3, 9]

In [7]:
def gen_all_suit_distributions():
    return every_split_in_4(total=13)



def every_hcpt_split_in_4_suits(score):

    all_splits = every_split_in_4(score)
    
    valid_splits = [spl for spl in all_splits if not (any(e for e in spl if e>10))]
    return list(reversed(valid_splits))
        

In [9]:
def single_split_in(num_splits, total):
    
    result = []
    
    if num_splits == 2:
                result = random.choice(_split(total))
    if num_splits == 3:
        result = random.choice(every_split_in_3(total))
        
    if num_splits == 4:
        result = random.choice(every_split_in_4(total))

        
    return result

In [92]:
def print_hand(hand):
    #print(hand)
    sp, he, di, cl = hand
    for s in sp:
        for h in he:
            for d in di:
                for c in cl:
                    #print(s,h,d,c)
                    print(f'{SPADES}{s[0]}{("-"*s[1])}', "  ",
                              f'{HEARTS}{h[0]}{("-"*h[1])}',"  ",
                              f'{DIAMONDS}{d[0]}{("-"*d[1])}', "  ",
                              f'{CLUBS}{c[0]}{("-"*c[1])}'
                             )

In [10]:
single_split_in(4, 8)

(3, 1, 1, 3)

In [11]:
all_distribs = gen_all_suit_distributions()
all_hcp_combos =every_split_in_4(40)

In [12]:
distrib, hcp = random.choice(all_distribs), random.choice(all_hcp_combos)
print(hcp)
for i in range(4):
    spts, hpts, dpts, cpts = generate_single_hand_points(hcp[i])
    print(spts, SPADES, hpts, HEARTS, dpts, DIAMONDS, cpts, CLUBS)

(11, 9, 4, 16)
2 ♠ 4 ♥ 0 ♦ 5 ♣
2 ♠ 4 ♥ 0 ♦ 3 ♣
4 ♠ 0 ♥ 0 ♦ 0 ♣
4 ♠ 10 ♥ 2 ♦ 0 ♣


In [13]:
generate_single_hand_points(18)

[1, 0, 9, 8]

In [14]:
[5, 4, 2, 2]

[5, 4, 2, 2]

In [59]:
for a in (['AE', 'ASD']):
    print(len(a))

2
3


In [117]:
hpdict = {10: ['AKQJ'], 9: ['AKQ'], 8:['AKJ'],
          7: ['AK', 'AQJ'], 6: ['AQ', 'KQJ'],
          5: ['AJ', 'KQ', ], 4: ['A', 'KJ'], 
          3: ['K', 'QJ'], 2:['Q'], 1:['J']}

def draw_high_cards_for(score, rnd=True):
    '''returns a combination of AKQJ that adds up to score'''
    
    if score > 10:
        ValueError("Requested score {score} not possible with AKQJ")
    
    if rnd:
        return random.choice(hpdict[score])
    else:
        return hpdict[score]

def gen_one_suit_for_one_hand(score, num_cards, rnd=True):
    """
    generates num_cards for one Suit that total up to Score HCP
    
    Returns a tuple with HC and LO cards
    Ex. Give me 5 cards of a single suit that add up to 7 HCP
    
    
    Example
    --------
    
    >>> gen_one_suit_for_one_hand(7,5)
    [('A', 4), ('Q', 4), ('J', 4)]
    
    If you want all possible HCP options, set rnd to False.
    
    >>> gen_one_suit_for_one_hand(7, 5, False)
    [('AK', 3), ('AQJ', 2)]
    
    """

    done = 0
    safety = 0
    while(not done):
        done = 1
        hcards = draw_high_cards_for(score, rnd)
        ret = []
        # Depending on rnd and the score, hcards could either be of length 1 or 2. 
        # If hcards is a list, we have to go through the list.
        # print(hcards, len(hcards), type(hcards))
        if isinstance(hcards, list):
            for hc in hcards:        
                if len(hc) > num_cards:
                    done = 0
                lo_cards = num_cards - len(hc)
                ret.append((hc, lo_cards))
                
        else: #case where hcards is a string
            if len(hcards) > num_cards:
                done = 0
            lo_cards = num_cards - len(hcards)
            ret.append((hcards, lo_cards))
            
        safety += 1
        if safety == 50:
            raise ValueError(f'Not possible to make high score {score} with {num_cards} cards')            

    
    return ret

In [118]:
gen_one_suit_for_one_hand(7,5, False)

[('AK', 3), ('AQJ', 2)]

In [120]:
def gen_every_hand(score=18, suits = [5, 4, 2, 2]):
    
    valid_pt_splits = every_hcpt_split_in_4_suits(score)
    
    all_hands = []
    for hand_pts in valid_pt_splits:
        hand, fail = [], 0
        for suit, suit_pt in enumerate(hand_pts):
            try:
                hs = gen_one_suit_for_one_hand(suit_pt, suits[suit], False) #tuple of cards and num_los's
                hand.append(hs)
            except:
                fail = 1
        if not fail:
            all_hands.append(hand)
            
        
    return all_hands

In [121]:
def gen_single_hand(score=18, suits = [5, 4, 2, 2]):
    
    valid_pt_splits = every_hcpt_split_in_4_suits(score)
    
    done = 0
    while not done:
        hand_pts = random.choice(valids)
        #print(f'hand_pts = {hand_pts}')
        hand = []
        done = 1
        for i, suit_pt in enumerate(hand_pts):
            try:
                hs = gen_one_suit_for_one_hand(suit_pt, suits[i], False) #tuple of cards and num_los's
                hand.append(hs)
            except:
                done = 0

    return hand

In [122]:
ah18_5422 = gen_every_hand()

In [123]:
len(ah18_5422)

205

In [124]:
for h in ah18_5422[:10]:
    print_hand(h)

♠AKQJ-    ♥AQ--    ♦J-    ♣J-
♠AKQJ-    ♥KQJ-    ♦J-    ♣J-
♠AKQJ-    ♥AJ--    ♦Q-    ♣J-
♠AKQJ-    ♥KQ--    ♦Q-    ♣J-
♠AKQJ-    ♥AJ--    ♦J-    ♣Q-
♠AKQJ-    ♥KQ--    ♦J-    ♣Q-
♠AKQJ-    ♥A---    ♦K-    ♣J-
♠AKQJ-    ♥A---    ♦QJ    ♣J-
♠AKQJ-    ♥KJ--    ♦K-    ♣J-
♠AKQJ-    ♥KJ--    ♦QJ    ♣J-
♠AKQJ-    ♥A---    ♦Q-    ♣Q-
♠AKQJ-    ♥KJ--    ♦Q-    ♣Q-
♠AKQJ-    ♥A---    ♦J-    ♣K-
♠AKQJ-    ♥A---    ♦J-    ♣QJ
♠AKQJ-    ♥KJ--    ♦J-    ♣K-
♠AKQJ-    ♥KJ--    ♦J-    ♣QJ
♠AKQJ-    ♥K---    ♦A-    ♣J-
♠AKQJ-    ♥K---    ♦KJ    ♣J-
♠AKQJ-    ♥QJ--    ♦A-    ♣J-
♠AKQJ-    ♥QJ--    ♦KJ    ♣J-
♠AKQJ-    ♥K---    ♦K-    ♣Q-
♠AKQJ-    ♥K---    ♦QJ    ♣Q-
♠AKQJ-    ♥QJ--    ♦K-    ♣Q-
♠AKQJ-    ♥QJ--    ♦QJ    ♣Q-
♠AKQJ-    ♥K---    ♦Q-    ♣K-
♠AKQJ-    ♥K---    ♦Q-    ♣QJ
♠AKQJ-    ♥QJ--    ♦Q-    ♣K-
♠AKQJ-    ♥QJ--    ♦Q-    ♣QJ
♠AKQJ-    ♥K---    ♦J-    ♣A-
♠AKQJ-    ♥K---    ♦J-    ♣KJ
♠AKQJ-    ♥QJ--    ♦J-    ♣A-
♠AKQJ-    ♥QJ--    ♦J-    ♣KJ
